# Restricted Boltzmann Machine

Importing Libraries

In [140]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [141]:
df_ratings = pd.read_csv("cleaned_ratings.csv", low_memory=False)
df_items = pd.read_csv("cleaned_items.csv", low_memory=False)

In [142]:
df_ratings = df_ratings[["user_id", "item_id", "rating"]]
df_ratings['rating'] = df_ratings['rating'].astype(int)


In [143]:
print(df_ratings.shape)
print(df_items.shape)

(42094, 3)
(999, 5)


In [144]:
df_items.head(2)

,id,title,description,imgurl,old_id
0,1,Cars,NaN,http://ecx.images-amazon.com/images/I/51pmW%2B...,B00000016T
1,2,Pet Sounds,NaN,http://ecx.images-amazon.com/images/I/51pDGkXj...,B00000016W


In [145]:
df_ratings.head(2)

,user_id,item_id,rating
0,1,206,3
1,1,9,5


In [146]:
df_items["ContinuousItemID"] = df_items.index
merged = df_items.merge(df_ratings, left_on="id", right_on="item_id").drop(["imgurl", "old_id", "description", "title"], axis=1)
merged.head(2)

,id,ContinuousItemID,user_id,item_id,rating
0,1,0,22,1,2
1,1,0,176,1,5


In [147]:
df_users = df_ratings.drop_duplicates(subset="user_id")

In [148]:
df_users.head(2)

,user_id,item_id,rating
0,1,206,3
4,2,757,4


In [149]:
df_users = df_users.rename(columns={'item_id': 'reduce_item_id', 'rating': 'reduce_rating'})
df_users["ContinuousUserID"] = df_users.index
merged = df_users.merge(merged, on="user_id").drop(["reduce_item_id", "reduce_rating", "id"], axis=1)

merged.head()

,user_id,ContinuousUserID,ContinuousItemID,item_id,rating
0,1,0,8,9,5
1,1,0,205,206,3
2,1,0,354,355,5
3,1,0,939,940,4
4,2,4,409,410,3


### Converting the ratings dataframe into a user-item Matrix

In [150]:
import scipy.sparse as sparse
import scipy.io


data = merged.rating
col = merged.ContinuousItemID
row = merged.ContinuousUserID

R = sparse.coo_matrix((data, (row, col))).tocsr()
print ('{0}x{1} user by movie matrix'.format(*R.shape))

42088x999 user by movie matrix


### Setting up the Restricted Boltzmann Machine

- A modified version is being used, as disucssed by Hinton
- The data has already been preprocessed above

The issue of missing values is taken by mapping each of the RBM nodes as as movie and take the ratings as input. Each user vector is taken as a training example.

### Building the Model

In [151]:
from __future__ import division
import tensorflow as tf


# visible layers set to the number of movies
# hidden units set to 20
n_visible,n_hidden = len(df_items), 20


In [152]:
#Specifying the structure
graph = tf.Graph()

with graph.as_default():
    v_bias = tf.placeholder(tf.float32, [n_visible])
    h_bias = tf.placeholder(tf.float32, [n_hidden])
    W = tf.placeholder(tf.float32, [n_visible, n_hidden])
    
    # visible to hidden pass
    v_1 = tf.placeholder(tf.float32, [None, n_visible])
    h_1_ = tf.sigmoid(tf.matmul(v_1, W) + h_bias)
    h_1 = tf.nn.relu(tf.sign(h_1_ - tf.random_uniform(tf.shape(h_1_))))
    
    
    # hidden to visible pass
    v_2_ = tf.sigmoid(tf.matmul(h_1, tf.transpose(W)) + v_bias)
    v_2 = tf.nn.relu(tf.sign(v_2_ - tf.random_uniform(tf.shape(v_2_))))
    h_2 = tf.nn.sigmoid(tf.matmul(v_2, W) + h_bias)
    
    # Learning rate
    lr = 0.01
    W_gradient_1 = tf.matmul(tf.transpose(v_1), h_1)
    W_gradient_2 = tf.matmul(tf.transpose(v_2), h_2)
    
    contrastive_divergence = ( W_gradient_1 - W_gradient_2 ) / tf.to_float(tf.shape(v_1)[0])
    
    # parameter updates
    W_update = W + lr * contrastive_divergence
    v_bias_update = v_bias + lr * tf.reduce_mean(v_1 - v_2, 0)
    h_bias_update = h_bias + lr * tf.reduce_mean(h_1 - h_2, 0)
    
    # error metrics
    mae = tf.reduce_mean(tf.abs(v_1 - v_2))
    rmse = tf.sqrt(tf.reduce_mean(tf.square(v_1 - v_2)))

### Training the Model

Specifying the number of epochs and batch sizes

In [153]:
import numpy as np


n_epoch = 20
batch_size = 100
current_W = np.zeros([n_visible, n_hidden], np.float32)
current_v_bias = np.zeros([n_visible], np.float32)
current_h_bias = np.zeros([n_hidden], np.float32)

Split into test and training sets

In [ ]:
# split into train and test
train_R = R[0:29461]
test_R = R[29461:]

# print(train_R.shape)
# print(test_R.shape)

Initializing the Machine

In [159]:
errors = []

start_time_total = time.time()

with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    
    
    for epoch in range(n_epoch):
#         start_time_epoch = time.time()
        for start in range(0, train_R.shape[0]-batch_size, batch_size):
            end = start + batch_size
            end = start + batch_size
            batch = train_R[start:end].todense()
            feed_dict = { v_1: batch, W: current_W, v_bias: current_v_bias, h_bias: current_h_bias }
            updates = [W_update, v_bias_update, h_bias_update]
            current_W, current_v_bias, current_h_bias = sess.run(updates, feed_dict=feed_dict)
    
#             print("\nTotal Epoch Time")
#             print(time.time() - start_time_epoch, "seconds")
    
        feed_dict = { v_1: test_R.todense(), W: current_W, v_bias: current_v_bias, h_bias: current_h_bias }
        mean_average_error, root_mean_squared_error = sess.run([mae, rmse], feed_dict=feed_dict)
        current_error = { "MAE": mean_average_error, "RMSE": root_mean_squared_error }
        
        print("MAE = {MAE:10.9f}, RMSE = {RMSE:10.9f}".format(**current_error))
        errors.append(current_error)

print("\nTotal Run Time")
print(time.time() - start_time_total, "seconds")

        
# plt.ylabel('Error')
# plt.xlabel('Epoch')
# plt.show()

MAE = 0.014073935, RMSE = 0.167520881
MAE = 0.014015539, RMSE = 0.167346179
MAE = 0.014009678, RMSE = 0.167361811
MAE = 0.013917404, RMSE = 0.167048305
MAE = 0.013886471, RMSE = 0.166959465
MAE = 0.013873619, RMSE = 0.166951522
MAE = 0.013825551, RMSE = 0.166812226
MAE = 0.013737614, RMSE = 0.166558057
MAE = 0.013757009, RMSE = 0.166596085
MAE = 0.013720294, RMSE = 0.166510478
MAE = 0.013710886, RMSE = 0.166490749
MAE = 0.013633093, RMSE = 0.166257605
MAE = 0.013638166, RMSE = 0.166216880
MAE = 0.013592199, RMSE = 0.166073963
MAE = 0.013551017, RMSE = 0.165936157
MAE = 0.013482738, RMSE = 0.165794984
MAE = 0.013515616, RMSE = 0.165866539
MAE = 0.013535932, RMSE = 0.165969729
MAE = 0.013518928, RMSE = 0.165917546
MAE = 0.013484683, RMSE = 0.165775642

Total Run Time
23.24080538749695 seconds


### Making Recommendations

In [160]:
from IPython.display import display, HTML